In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

c:\Users\PC\miniconda3\envs\tf-cpu\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\PC\miniconda3\envs\tf-cpu\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
path_to_file = keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

In [3]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print('Length of text: {} charachters'.format(len(text)))

Length of text: 1115394 charachters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



ENCODING

In [5]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return "".join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length = 100
example_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Building Model

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    inputs = tf.keras.Input(batch_shape=(batch_size, None), dtype=tf.int32)
    x = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer="glorot_uniform"
    )(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (64, None)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (64, None, 256)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 65)         │        66,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_prediction = model(input_example_batch)
    print(example_batch_prediction.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
print(len(example_batch_prediction))
print(example_batch_prediction)

64
tf.Tensor(
[[[-4.61240811e-03  1.63966487e-03 -1.05109706e-03 ...  2.60368688e-05
    4.04567830e-03 -2.53483769e-04]
  [-3.94372316e-03  6.34742249e-03 -2.35980679e-03 ...  4.11759689e-03
    3.92765878e-03  2.69060908e-03]
  [-5.00849215e-04  7.04597635e-03 -2.49842973e-03 ...  3.68200848e-03
    6.24716165e-04  1.85573660e-03]
  ...
  [ 6.15436584e-05 -2.22764982e-04 -2.26923823e-03 ...  3.25377937e-03
   -1.63132045e-03 -2.16826261e-03]
  [ 4.54987725e-03  2.57715047e-03 -9.48888250e-04 ...  4.38893773e-03
   -4.46687220e-03 -3.32914805e-03]
  [ 1.37498137e-04  4.31576325e-03 -8.98921979e-04 ...  4.00550663e-03
    2.79717147e-04 -2.84965639e-03]]

 [[-3.62777081e-03 -1.49623584e-03 -6.32101484e-03 ... -2.87026842e-03
    2.02416000e-03 -2.92362133e-03]
  [-1.46306003e-03  1.03005383e-03 -5.21266740e-03 ... -3.24819470e-04
   -1.94050046e-03 -2.40783719e-03]
  [ 3.05549521e-03 -3.07170581e-03 -8.23425595e-03 ... -2.71718367e-03
   -1.09741138e-03  2.33151019e-04]
  ...
  [ 1.828

In [16]:
pred = example_batch_prediction[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-4.6124081e-03  1.6396649e-03 -1.0510971e-03 ...  2.6036869e-05
   4.0456783e-03 -2.5348377e-04]
 [-3.9437232e-03  6.3474225e-03 -2.3598068e-03 ...  4.1175969e-03
   3.9276588e-03  2.6906091e-03]
 [-5.0084922e-04  7.0459764e-03 -2.4984297e-03 ...  3.6820085e-03
   6.2471617e-04  1.8557366e-03]
 ...
 [ 6.1543658e-05 -2.2276498e-04 -2.2692382e-03 ...  3.2537794e-03
  -1.6313205e-03 -2.1682626e-03]
 [ 4.5498773e-03  2.5771505e-03 -9.4888825e-04 ...  4.3889377e-03
  -4.4668722e-03 -3.3291481e-03]
 [ 1.3749814e-04  4.3157632e-03 -8.9892198e-04 ...  4.0055066e-03
   2.7971715e-04 -2.8496564e-03]], shape=(100, 65), dtype=float32)


In [17]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-4.6124081e-03  1.6396649e-03 -1.0510971e-03 -1.5650725e-04
 -3.3166262e-03 -9.0640126e-04 -4.2850757e-03 -3.2226468e-05
  6.1755901e-04 -4.4053519e-04  3.2433384e-04 -3.4911418e-03
  7.7201463e-03  3.4157098e-03  5.5186269e-03 -9.7741373e-05
 -3.8897966e-03 -3.5258872e-03  1.4725479e-03  1.5224860e-03
  2.5860462e-03 -1.9491153e-03 -2.7618646e-03 -5.4274793e-03
  3.7451603e-03 -3.7883647e-04  5.1989034e-04 -3.3908235e-03
 -1.8078517e-03  1.4428380e-03  5.0310940e-03 -5.7438028e-04
 -3.7095330e-03  2.8856453e-03  1.1415258e-03  3.3774534e-03
  3.4724805e-03  2.3394046e-04 -2.1370919e-03  2.4270252e-03
 -2.2422615e-03  5.5239582e-03  3.8640413e-03  4.5099999e-03
 -8.3406165e-04  1.3464985e-03  3.4763146e-04 -8.5753680e-04
  5.7834345e-03 -3.3733258e-03  1.3560931e-03  6.2963821e-04
 -3.6314065e-03  1.4660950e-04  1.8513033e-03 -3.5593184e-03
 -4.4179778e-03  5.9663528e-04  3.1463001e-03 -3.6286127e-03
  4.6372376e-04 -4.0848460e-04  2.6036869e-05  4.0456783e-03
 -2.534837

In [18]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
prediction_chars = int_to_text(sampled_indices)

prediction_chars

"blDTW$in'e,,DpglOfCLXlfcz;bU3baBmxlvSTRNTgYvEEGn!$3stmOXWcp-n;ZdMYTUghjtN3cNlP\nSsY!\nScE&kVbDkJDz,RHv"

In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer="adam", loss=loss)

Checkpoints

In [21]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only=True
)

Training

In [22]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 144s 827ms/step - loss: 2.4070
Epoch 2/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 143s 828ms/step - loss: 1.7749
Epoch 3/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 143s 829ms/step - loss: 1.5656
Epoch 4/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 137s 793ms/step - loss: 1.4594
Epoch 5/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 139s 803ms/step - loss: 1.3934
Epoch 6/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 146s 843ms/step - loss: 1.3460
Epoch 7/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 141s 816ms/step - loss: 1.3071
Epoch 8/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 144s 836ms/step - loss: 1.2713
Epoch 9/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 137s 794ms/step - loss: 1.2402
Epoch 10/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 142s 824ms/step - loss: 1.2091
Epoch 11/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 139s 803ms/step - loss: 1.1776
Epoch 12/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 138s 801ms/step - loss: 1.1458
Epoch 13/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 138s 801ms/step - loss: 1.1130
Epoch 14/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 145s 841ms/step - loss: 1.0797
E

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [25]:
model.load_weights('./training_checkpoints/ckpt_10.weights.h5')
model.build(tf.TensorShape([1, None]))

In [27]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

AttributeError: 'Functional' object has no attribute 'reset_states'